In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing # Not sure yet if I need this
import bw_migrations # Not sure yet if I need this
from bw2io.importers.json_ld_lcia import JSONLDLCIAImporter

import os
import numpy as np
import pandas as pd

bd.projects.set_current('USLCI_FY22_Q2_3')

path = r'databases/FY22_Q2_01_Zolca_LCIA_methods_mapping_FEDEFL_3'
uslci = bi.importers.JSONLDImporter(
    path, 
    "USLCI_FY22_Q2_1", 
    preferred_allocation="PHYSICAL_ALLOCATION" # Most USLCI allocations are based on physical allocations (source: Rebe Feraldi)
)

# Fix openLCA bug: on OpenLCA, on some processes (i.e. Natural Gas), the output exchanges there is a column named 
# 'Avoided product', and when it saves into JSON it sets the exchange['input'] to True instead of to False
for process_key, process_values in uslci.data['processes'].items():
    for i, exchange in enumerate(process_values['exchanges']):
        if (exchange['avoidedProduct'] == True) & (exchange['input'] == True):
            print(exchange['input'], exchange['flow']['name'])
            exchange['input'] = False
            uslci.data['processes'][process_key]['exchanges'][i] = exchange

uslci.apply_strategies()

uslci.apply_strategy(bi.strategies.special.add_dummy_processes_and_rename_exchanges)

uslci.write_separate_biosphere_database()
uslci.write_database()
uslci_methods = bi.importers.JSONLDLCIAImporter(path)
uslci_methods.apply_strategies()
uslci_methods.match_biosphere_by_id('USLCI_FY22_Q2_1 biosphere')
uslci_methods.statistics()
method_key = [act for act in bd.methods if 'Global warming' in str(act) and 'TRACI' in str(act)][0]
my_method = bd.Method(method_key)
alu = [act for act in db if act['name'].lower().startswith('aluminium, extrusion')][0]
functional_unit = {alu : 5}
lca = bc.LCA(functional_unit, method_key) 
lca.lci()  # Fails here   < -- FAIL 
lca.lcia()
lca.score